In [1]:

import numpy as np, scipy as sc, pandas as pd
from datetime import datetime 


In [2]:

print(datetime.now())


2022-03-27 13:45:09.450403


In [3]:

global inpdf, inpdf_strategy, data_type


In [4]:

data_type = 'place'


In [5]:

%run ./01_GetData.ipynb


importing:	D:\GDrive\Analyticsflex\Racing\Data_dwbfprices\Place\dwbfgreyhoundplace01012019.csv
importing:	D:\GDrive\Analyticsflex\Racing\Data_dwbfprices\Place\dwbfgreyhoundplace01012020.csv
importing:	D:\GDrive\Analyticsflex\Racing\Data_dwbfprices\Place\dwbfgreyhoundplace01012021.csv
importing:	D:\GDrive\Analyticsflex\Racing\Data_dwbfprices\Place\dwbfgreyhoundplace01012022.csv
importing:	D:\GDrive\Analyticsflex\Racing\Data_dwbfprices\Place\dwbfgreyhoundplace01022019.csv
importing:	D:\GDrive\Analyticsflex\Racing\Data_dwbfprices\Place\dwbfgreyhoundplace01022020.csv
importing:	D:\GDrive\Analyticsflex\Racing\Data_dwbfprices\Place\dwbfgreyhoundplace01022021.csv
importing:	D:\GDrive\Analyticsflex\Racing\Data_dwbfprices\Place\dwbfgreyhoundplace01022022.csv
importing:	D:\GDrive\Analyticsflex\Racing\Data_dwbfprices\Place\dwbfgreyhoundplace01032018.csv
importing:	D:\GDrive\Analyticsflex\Racing\Data_dwbfprices\Place\dwbfgreyhoundplace01032019.csv
importing:	D:\GDrive\Analyticsflex\Racing\Data_dwb

In [8]:

print(datetime.now())


2022-03-27 13:46:44.069085


In [ ]:

# print(inpdf.shape)
# print(inpdf.columns.values.tolist)

inpdf.head()


In [14]:

#inpdf.groupby('EVENT_ID').apply(lambda grp: grp['a'].sum() if grp['b'].sum() > 1 else grp['a'].max()) 
#BSP_min = inpdf.groupby('EVENT_ID').apply(lambda grp: 1 if grp['BSP'] == grp['BSP'].min() else 0 ) 


In [10]:

# Strategy (WIP) - to Place all the top 3 or top 2 greyhounds
'''
print(datetime.now())
inpdf_top3 = inpdf.sort_values(['EVENT_ID','BSP']).groupby('EVENT_ID').apply(pd.DataFrame.head, n=3) 
print(datetime.now())
# runtime = 10 min
'''


2022-03-27 13:47:04.618556
2022-03-27 13:53:14.480801


In [26]:


#inpdf_top3[inpdf_top3.BSP.isna()].groupby('dt').agg({'EVENT_ID':'nunique'})
#inpdf_top3[inpdf_top3.datetime.dt.year > 2018].sort_index().head(5)



In [29]:

# Bet the fav strategy 
if data_type == 'win':
    BSP_min = inpdf.groupby('EVENT_ID')['BSP'].min().reset_index().rename( columns = {'BSP':'BSP_min'} ) 
    inpdf_interim = inpdf.merge(BSP_min, on = 'EVENT_ID')

# Place the fav strategy 
if data_type == 'place':
    rc_count = inpdf.groupby('EVENT_ID')['SELECTION_NAME'].count().reset_index().rename( columns = {'SELECTION_NAME':'racers'} ) 
    BSP_min = inpdf.groupby('EVENT_ID')['BSP'].min().reset_index().rename( columns = {'BSP':'BSP_min'} ) 
    print(inpdf.shape)
    inpdf_interim = inpdf.merge(rc_count, on = 'EVENT_ID')
    print(inpdf_interim.shape)
    inpdf_interim_vf = inpdf_interim.merge(BSP_min, on = 'EVENT_ID')
    print(inpdf_interim_vf.shape)
    del inpdf_interim


(1800500, 20)
(1800500, 21)
(1800500, 22)


In [30]:

print(datetime.now())

inpdf_interim_vf['hit'] = inpdf_interim_vf.apply(lambda x : 1 if ( x['BSP'] == x['BSP_min']) & (x.WIN_LOSE ==1) else 0, axis = 1) 
inpdf_interim_vf['prof'] = inpdf_interim_vf.apply(lambda x : x['BSP'] -1 if ( x['BSP'] == x['BSP_min'] ) & ( x.WIN_LOSE ==1 ) else ( -1 if ( x['BSP'] == x['BSP_min'] ) & ( x.WIN_LOSE == 0 ) else 0 ), axis = 1) 

print(datetime.now())


In [36]:

#inpdf_interim.head(25)
inpdf_interim_vf.tail(5)


,EVENT_ID,EVENT_NAME,EVENT_DT,SELECTION_ID,SELECTION_NAME,WIN_LOSE,BSP,PPWAP,MORNINGWAP,PPMAX,...,PPTRADEDVOL,IPTRADEDVOL,datetime,dt,rc_ctry,rc_track,racers,BSP_min,hit,prof
1800495,192869819,TO BE PLACED,30-12-2021 20:36,39597307,3. Another Dove,1.0,3.100000,1,1.0,1.0,...,1049.76,0.0,2021-12-30 20:36:00,2021-12-30,TRP,Perry Barr 30th Dec,6,1.796509,0,0.0
1800496,192869819,TO BE PLACED,30-12-2021 20:36,39301720,4. Moyar Echo,0.0,2.795171,1,1.0,1.0,...,926.16,0.0,2021-12-30 20:36:00,2021-12-30,TRP,Perry Barr 30th Dec,6,1.796509,0,0.0
1800497,192869819,TO BE PLACED,30-12-2021 20:36,35447585,6. Dolla Wisecrack,0.0,2.368171,1,1.0,1.0,...,1665.62,0.0,2021-12-30 20:36:00,2021-12-30,TRP,Perry Barr 30th Dec,6,1.796509,0,0.0
1800498,192869819,TO BE PLACED,30-12-2021 20:36,39382771,2. Coill Bhui Rebel,0.0,4.758659,1,1.0,1.0,...,834.72,0.0,2021-12-30 20:36:00,2021-12-30,TRP,Perry Barr 30th Dec,6,1.796509,0,0.0
1800499,192869819,TO BE PLACED,30-12-2021 20:36,41554859,5. Ocean Ringo,0.0,6.689477,1,1.0,1.0,...,452.88,0.0,2021-12-30 20:36:00,2021-12-30,TRP,Perry Barr 30th Dec,6,1.796509,0,0.0


In [37]:

# Hit Rate
inpdf_interim_vf['hit'].sum() / inpdf_interim_vf['EVENT_ID'].nunique()


0.5727738346307183

In [38]:

#prof
inpdf_interim_vf['prof'].sum()


7314.009973772436

In [24]:

inpdf_strategy = inpdf_interim_vf
